#### 목표 
> * 사전학습 모델인 Backbone 모델의 종류와 개념을 알고, Transfer Learning의 개념을 설명할 수 있다.
> * VGG, ResNet과 같은 기본적인 Backbone 모델을 불러와서 사용할 수 있다.
> * Backbone 모델을 원하는 레이어(layer)만큼 새로 학습시켜서 사용할 수 있다.
> * Backbone 모델을 Transfer Learning 시킴으로써 원하는 이미지를 분류시킬 수 있다.
> * 이미 잘 학습되어 있는 모델을 가지고 이미지를 분류할 수 있다.


---

## 1. 강아지 고양이 분류기 : 모델 직접 설계
#### tensorflow_datasets 데이터를 이용

```Python
import warnings
warnings.filterwarnings("ignore")
# 오류 무시


import tensorflow as tf
import tensorflow_datasets as tfds
# 텐서플로우 데이터셋 로드(cats_vs_dog 활용, 786.68 MiB, 23262장)


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# 시각화를 위한 matplotlib. 선명하게 하기 위해 retina 문장 추가


(raw_train, raw_validation, raw_test), metadata = tfds.load(
    name='cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    data_dir='~/Dataset/cats_vs_dogs/',
    download=False,
    with_info=True,
    as_supervised=True,
)
# 데이터 스플릿


print(raw_train)
print(raw_validation)
print(raw_test)
# 데이터 확인


plt.figure(figsize=(10, 5))

get_label_name = metadata.features['label'].int2str

for idx, (image, label) in enumerate(raw_train.take(10)):  # 10개의 데이터를 가져 옵니다.
    plt.subplot(2, 5, idx+1)
    plt.imshow(image)
    plt.title(f'label {label}: {get_label_name(label)}')
    plt.axis('off')
# 10장의 데이터 이미지 확인
# 강아지는 label 1로, 고양이는 label 0으로 설정되어 있음.


IMG_SIZE = 160 # 리사이징할 이미지의 크기

def format_example(image, label):
    image = tf.cast(image, tf.float32)  # image=float(image)같은 타입캐스팅의  텐서플로우 버전입니다.
    image = (image/127.5) - 1 # 픽셀값의 scale 수정
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label
# 이미지 포맷 일치를 위한 함수.(타입캐스팅)


IMG_SIZE = 160 # 리사이징할 이미지의 크기

def format_example(image, label):
    image = tf.cast(image, tf.float32)  # image=float(image)같은 타입캐스팅의 텐서플로우 버전.
    image = (image/127.5) - 1 # 픽셀값의 scale 수정(-1~1 사이의 실수값으로)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label
# (160,160) 픽셀로 통일 및 각 픽셀 데이터값을 scaling


train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)
# train, validataion, test 데이터셋에 map함수를 이용해 적용


print(train)
print(validation)
print(test)
# 적용이 잘 됐는지 확인(160, 160, 3)


plt.figure(figsize=(10, 5))


get_label_name = metadata.features['label'].int2str

for idx, (image, label) in enumerate(train.take(10)):
    plt.subplot(2, 5, idx+1)
    image = (image + 1) / 2
    plt.imshow(image)
    plt.title(f'label {label}: {get_label_name(label)}')
    plt.axis('off')
# 이미지를 불러와 확인


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D


model = Sequential([
    Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=(160, 160, 3)),
    MaxPooling2D(),
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(units=512, activation='relu'),
    Dense(units=2, activation='softmax')
])
# 본격적인 모델 설계


model.summary()
# 모델 구조 확인


learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])
# 모델 compile 설정(optimizer, loss, metrics)


BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000
# batch size와 shuffle buffer size 설정


train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)
# BATCH_SIZE에 따라 32개의 데이터를 랜덤으로 뿌려줄 train_batches, validation_batches, test_batches를 생성


validation_steps = 20
loss0, accuracy0 = model.evaluate(validation_batches, steps=validation_steps)

print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))
# 학습 전 초기 모델을 만들어서 평가
# validation(검증)을 하기 위한 데이터셋인 validation_batches를 이용해 20번의 예측을 해 보고, 평균 loss와 평균 accuracy를 확인할 것.
# => loss는 0.7, accuracy는 약 50%.


EPOCHS = 10
history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)
# 학습 시작
# => 훈련 데이터셋에 대한 정확도(accuracy)는 약 90% 남짓, 검증 데이터셋에 대한 accuracy(val_accuracy)는 약 80% 조금 못 미치게 나옴


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()
# 결과 시각화
# validation accuracy를 보면 overfitting이 일어남을 알 수 있음


for image_batch, label_batch in test_batches.take(1):
    images = image_batch
    labels = label_batch
    predictions = model.predict(image_batch)
    break

predictions
# 모델의 예측 결과를 확인
# [고양이일 확률, 강아지의 확률] // [1.0, 0.0]에 가까울수록 label이 0인 고양이로, [0.0, 1.0]에 가까울수록 label이 1인 강아지로 예측


predictions = np.argmax(predictions, axis=1)
predictions
# prediction 값들을 실제 추론한 라벨(0:고양이, 1:강아지)로 변환.


plt.figure(figsize=(20, 12))

for idx, (image, label, prediction) in enumerate(zip(images, labels, predictions)):
    plt.subplot(4, 8, idx+1)
    image = (image + 1) / 2
    plt.imshow(image)
    correct = label == prediction
    title = f'real: {label} / pred :{prediction}\n {correct}!'
    if not correct:
        plt.title(title, fontdict={'color': 'red'})
    else:
        plt.title(title, fontdict={'color': 'blue'})
    plt.axis('off')
# 예측 결과를 시각화


count = 0   # 정답을 맞춘 개수
for image, label, prediction in zip(images, labels, predictions):
    # [[YOUR CODE]]
    if label.numpy() == prediction:
        #맞았다
        count += 1
        

print(count / 32 * 100)
# 예측 결과가 얼마나 맞았는지 확인
```

#### model compile에서 정해야 할 3가지 요소
 * **optimizer**는 학습을 어떤 방식으로 시킬 것인지 결정. 어떻게 최적화시킬 것인지를 결정하기 때문에 최적화 함수라고 부르기도 함.
 * **loss**는 모델이 학습해나가야 하는 방향을 결정. 이 문제에서는 모델의 출력은 입력받은 이미지가 고양이인지 강아지인지에 대한 확률분포로 두었으므로, 입력 이미지가 고양이(label=0)일 경우 모델의 출력이 [1.0, 0.0]에 가깝도록, 강아지(label=1)일 경우 [0.0, 1.0]에 가까워지도록 하는 방향을 제시할 것.(sparce categorical crossentropy 사용)
 * **metrics**는 모델의 성능을 평가하는 척도. 분류 문제를 풀 때, 성능을 평가할 수 있는 지표인 정확도(accuracy), 정밀도(precision), 재현율(recall) 등을 사용.(여기서는 정확도를 사용)

## 2. 강아지 고양이 분류기 : 전이 학습(Transfer Learning)
#### VGG16 모델을 가져와서 적용할 것

```Python
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
# 모델에서 이미지가 어떻게 변하는지 알아보자


base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                         include_top=False,
                                         weights='imagenet')
# pre-trained 모델인 VGG16 가져오기


image_batch.shape
# 적용하기 전 image_batch의 원래 사이즈를 다시 확인


feature_batch = base_model(image_batch)
feature_batch.shape
# 모델을 적용한 결과물의 shape은 height, width는 각각 5로 매우 작아졌고, 512로 channel이 늘어났음


base_model.summary()
# 모델 구조 확인
```

#### 본격적인 모델 설계

```Python

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(512, activation='relu')
prediction_layer = tf.keras.layers.Dense(2, activation='softmax')
prediction_batch = prediction_layer(dense_layer(feature_batch_average))  
# 모델에 들어갈 Layer


base_model.trainable = False
# VGG16에 해당하는 base_model은 학습시키지 않을 예정이므로 학습 여부를 결정하는 trainable 변수를 False로 지정.


model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  dense_layer,
  prediction_layer
])
# 모델 설계


model.summary()
# 모델 확인


base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])
# model compile


validation_steps=20
loss0, accuracy0 = model.evaluate(validation_batches, steps = validation_steps)

print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))
# validation 데이터로 초기 model 학습을 한 뒤 확인.
# 정확도가 50% 정도 나옴


EPOCHS = 5   # 이번에는 이전보다 훨씬 빠르게 수렴되므로 5Epoch이면 충분합니다.

history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)
# 모델 학습


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()
# 시각화
# 이미 훈련된 모델을 사용하자 직접 설계한 모델보다 좋은 결과.(정확도와 validation loss 모두 안정적)


for image_batch, label_batch in test_batches.take(1):
    images = image_batch
    labels = label_batch
    predictions = model.predict(image_batch)
    pass

predictions
# test sample 32개로 prediction 생성
# 마찬가지로 0~1 사이 확률값으로 나옴


import numpy as np
predictions = np.argmax(predictions, axis=1)
predictions
# 실제 추론한 라벨(0:고양이, 1:강아지)로 변환


plt.figure(figsize=(20, 12))

for idx, (image, label, prediction) in enumerate(zip(images, labels, predictions)):
    plt.subplot(4, 8, idx+1)
    image = (image + 1) / 2
    plt.imshow(image)
    correct = label == prediction
    title = f'real: {label} / pred :{prediction}\n {correct}!'
    if not correct:
        plt.title(title, fontdict={'color': 'red'})
    else:
        plt.title(title, fontdict={'color': 'blue'})
    plt.axis('off')
# 예측 결과물 시각화


plt.figure(figsize=(20, 12))

for idx, (image, label, prediction) in enumerate(zip(images, labels, predictions)):
    plt.subplot(4, 8, idx+1)
    image = (image + 1) / 2
    plt.imshow(image)
    correct = label == prediction
    title = f'real: {label} / pred :{prediction}\n {correct}!'
    if not correct:
        plt.title(title, fontdict={'color': 'red'})
    else:
        plt.title(title, fontdict={'color': 'blue'})
    plt.axis('off')
# 예측 정확도 확인
```

---
# 3. 모델에 checkpoint 적용 / 원하는 이미지를 넣어서 예측 결과 도출
(미리 checkpoint 폴더와 원하는 이미지가 들어간 images 폴더를 만들어 두었음)<br><br>



#### checkpoint 생성

```Python
import os

checkpoint_dir = os.getenv("HOME") + "/Aiffel_Project/Dataset/cat_vs_dog/checkpoint"
checkpoint_file_path = os.path.join(checkpoint_dir, 'checkpoint')

if not os.path.exists('checkpoint_dir'):
    os.mkdir('checkpoint_dir')
    
model.save_weights(checkpoint_file_path)     # checkpoint 파일 생성

if os.path.exists(checkpoint_file_path):
  print('checkpoint 파일 생성 OK')
```

#### 원하는 이미지를 모델에 적용해서 판별을 하는지 확인

```Python
from tensorflow.keras.preprocessing.image import load_img, img_to_array
# load_img, img_to_array 함수를 가져와야 함


IMG_SIZE = 160
img_dir_path = os.getenv("HOME") + "/Aiffel_Project/Dataset/cat_vs_dog/checkpoint"
dog_image_path = os.path.join(img_dir_path, 'my_dog.jpg')

dog_image = load_img(dog_image_path, target_size=(IMG_SIZE, IMG_SIZE))
dog_image
# 모델이 (160, 160) size만 적용 가능하므로 파라미터 입력하기
# 강아지 이미지가 잘 적용이 됐는지 확인


dog_image = img_to_array(dog_image).reshape(1, IMG_SIZE, IMG_SIZE, 3)
dog_image.shape
# 이미지를 array(배열) 자료형으로 변경


prediction = model.predict(dog_image)
prediction
# 예측


def show_and_predict_image(dirpath, filename, img_size=160):
    filepath = os.path.join(dirpath, filename)
    image = load_img(filepath, target_size=(img_size, img_size))
    plt.imshow(image)
    plt.axis('off')
    image = img_to_array(image).reshape(1, img_size, img_size, 3)
    prediction = model.predict(image)[0]
    cat_percentage = round(prediction[0] * 100)
    dog_percentage = round(prediction[1] * 100)
    print(f"This image seems {dog_percentage}% dog, and {cat_percentage}% cat.")
# 모델에 넣어서 예측 결과를 알려주는 함수를 생성    


filename = 'my_dog.jpg'

show_and_predict_image(img_dir_path, filename)
# my_dog.jpg를 넣어서 함수가 잘 내놓는지 확인


filename = 'my_cat.jpg'

show_and_predict_image(img_dir_path, filename)
# my_cat.jpg
```